In [1]:
import os

In [3]:
%pwd

'/home/lucky/Documents/data_science_interview/Udemy/Chicken-Disease-Classification/research'

In [4]:
os.chdir("../")

In [5]:
pwd

'/home/lucky/Documents/data_science_interview/Udemy/Chicken-Disease-Classification'

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Data Ingestion Configuration Manager

In [21]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [22]:
CONFIG_FILE_PATH, PARAMS_FILE_PATH

(PosixPath('config/config.yaml'), PosixPath('params.yaml'))

In [14]:
cnf = read_yaml(CONFIG_FILE_PATH)

[2023-12-07 19:22:48,521: INFO: common: yaml file: config/config.yaml loaded successfully]


In [18]:
cnf.data_ingestion

ConfigBox({'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/luckyIITR/datarepo/raw/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'})

In [23]:
class ConfigurationManager:
    """
        responsible for creating the Directories needed for data
        and return entity -> DataIngestionConfig
    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# Data Ingestion Class

In [24]:
import os
import urllib.request as request
import ssl
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        # Create an unverified SSL context
        ssl._create_default_https_context = ssl._create_unverified_context
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-07 19:54:04,995: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-07 19:54:04,997: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-07 19:54:04,998: INFO: common: created directory at: artifacts]
[2023-12-07 19:54:04,999: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-07 19:54:08,824: INFO: 2944401693: artifacts/data_ingestion/data.zip download! with following info: 
Date: Thu, 07 Dec 2023 14:24:07 GMT
Content-Type: application/zip
Content-Length: 11616915
Connection: close
Cache-Control: max-age=300
Content-Security-Policy: default-src *.opendns.com gateway.id.swg.umbrella.com 'none'; style-src *.opendns.com gateway.id.swg.umbrella.com 'unsafe-inline'; sandbox;
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BC64:181AD0:123A8F:16C524:657